In [ ]:
! pip install datasets

In [ ]:
from datasets import load_dataset

# load the dataset from huggingface in streaming mode and shuffle it
wiki_data = load_dataset(
    'vblagoje/wikipedia_snippets_streamed',
    split='train',
    streaming=True
).shuffle(seed=960)

In [ ]:
# filter only documents with History as section_title
history = wiki_data.filter(
    lambda d: d['section_title'].startswith('Technology')
)

In [ ]:
history

In [ ]:
!pip install tqdm

In [ ]:
from tqdm.auto import tqdm  # progress bar

total_doc_count = 500

counter = 0
docs = []
# iterate through the dataset and apply our filter
for d in tqdm(history, total=total_doc_count):
    # extract the fields we need
    doc = {
        "article_title": d["article_title"],
        "section_title": d["section_title"],
        "passage_text": d["passage_text"]
    }
    # add the dict containing fields we need to docs list
    docs.append(doc)

    # stop iteration once we reach 50k
    if counter == total_doc_count:
        break

    # increase the counter on every iteration
    counter += 1

  0%|          | 0/500 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer()

In [ ]:
passage_texts = [doc['passage_text'] for doc in docs]

# Create a tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(passage_texts)

In [ ]:
unique_words_count = len(tokenizer.word_index)

print("Number of unique words:", unique_words_count)

Number of unique words: 4830


In [ ]:
input_sequences = []

# Iterate through each passage_text in the list
for passage_text in passage_texts:
    # Tokenize the passage_text
    tokenized_sentence = tokenizer.texts_to_sequences([passage_text])[0]

    # Generate input sequences by considering prefixes of tokenized_sentence
    for i in range(1, len(tokenized_sentence)):
        input_sequences.append(tokenized_sentence[:i+1])

In [ ]:
input_sequences

[[3, 937],
 [3, 937, 188],
 [3, 937, 188, 31],
 [3, 937, 188, 31, 2176],
 [3, 937, 188, 31, 2176, 12],
 [3, 937, 188, 31, 2176, 12, 938],
 [3, 937, 188, 31, 2176, 12, 938, 73],
 [3, 937, 188, 31, 2176, 12, 938, 73, 13],
 [3, 937, 188, 31, 2176, 12, 938, 73, 13, 4],
 [3, 937, 188, 31, 2176, 12, 938, 73, 13, 4, 1326],
 [3, 937, 188, 31, 2176, 12, 938, 73, 13, 4, 1326, 5],
 [3, 937, 188, 31, 2176, 12, 938, 73, 13, 4, 1326, 5, 2177],
 [3, 937, 188, 31, 2176, 12, 938, 73, 13, 4, 1326, 5, 2177, 3],
 [3, 937, 188, 31, 2176, 12, 938, 73, 13, 4, 1326, 5, 2177, 3, 939],
 [3, 937, 188, 31, 2176, 12, 938, 73, 13, 4, 1326, 5, 2177, 3, 939, 511],
 [3, 937, 188, 31, 2176, 12, 938, 73, 13, 4, 1326, 5, 2177, 3, 939, 511, 2],
 [3,
  937,
  188,
  31,
  2176,
  12,
  938,
  73,
  13,
  4,
  1326,
  5,
  2177,
  3,
  939,
  511,
  2,
  213],
 [3,
  937,
  188,
  31,
  2176,
  12,
  938,
  73,
  13,
  4,
  1326,
  5,
  2177,
  3,
  939,
  511,
  2,
  213,
  940],
 [3,
  937,
  188,
  31,
  2176,
  12,
  93

In [ ]:
max_len = max([len(x) for x in input_sequences ])
max_len

115

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_input_sequences = pad_sequences(input_sequences, maxlen = max_len, padding = 'pre')

In [ ]:
padded_input_sequences

array([[  0,   0,   0, ...,   0,   3, 937],
       [  0,   0,   0, ...,   3, 937, 188],
       [  0,   0,   0, ..., 937, 188,  31],
       ...,
       [  0,   0,   0, ..., 117,   6,   1],
       [  0,   0,   0, ...,   6,   1, 330],
       [  0,   0,   0, ...,   1, 330,   2]], dtype=int32)

In [ ]:
X = padded_input_sequences[:,:-1]

In [ ]:
y = padded_input_sequences[:,-1]

In [ ]:
len(padded_input_sequences)

19210

In [ ]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes = 19211)

In [ ]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
model = Sequential()
model.add(Embedding(19211, 150, input_length =  114))

In [ ]:
model.add(LSTM(150))
model.add(Dense(19211, activation = 'softmax'))

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 114, 150)          2881650   
                                                                 
 lstm_1 (LSTM)               (None, 150)               180600    
                                                                 
 dense_1 (Dense)             (None, 19211)             2900861   
                                                                 
Total params: 5,963,111
Trainable params: 5,963,111
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X,y, epochs = 100)

Epoch 1/100
 15/601 [..............................] - ETA: 5:04 - loss: 0.2471 - accuracy: 0.9792

KeyboardInterrupt: ignored

In [ ]:
text = "logic"

tokenized_text = tokenizer.texts_to_sequences([text])[0]

In [ ]:
padded_token_text = pad_sequences([tokenized_text], maxlen = 114, padding = 'pre')

In [ ]:
padded_token_text

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0, 1828]], dtype=int32)

In [ ]:
model.predict(padded_token_text)

1/1 [==============================] - 0s 35ms/step


array([[1.3752584e-10, 4.5193069e-02, 4.7626084e-01, ..., 1.2494267e-10,
        1.4039646e-10, 1.0764426e-10]], dtype=float32)

In [ ]:
import numpy as np
max_prob_word_embed = np.argmax(model.predict(padded_token_text))

1/1 [==============================] - 0s 54ms/step


In [ ]:
max_prob_word_embed

2

In [ ]:
for word, index in tokenizer.word_index.items():
  if index == max_prob_word_embed:
    print(word)

of


In [ ]:
text = "financial"
for i in range(15):
  tokenized_text = tokenizer.texts_to_sequences([text])[0]
  padded_token_text = pad_sequences([tokenized_text], maxlen = 114, padding = 'pre')
  max_prob_word_embed = np.argmax(model.predict(padded_token_text))
  for word, index in tokenizer.word_index.items():
      if index == max_prob_word_embed:
         text = text + " " + word
         print(text)

1/1 [==============================] - 0s 38ms/step
financial compliance
1/1 [==============================] - 0s 34ms/step
financial compliance funding
1/1 [==============================] - 0s 36ms/step
financial compliance funding as
1/1 [==============================] - 0s 39ms/step
financial compliance funding as the
1/1 [==============================] - 0s 32ms/step
financial compliance funding as the technology
1/1 [==============================] - 0s 36ms/step
financial compliance funding as the technology to
1/1 [==============================] - 0s 35ms/step
financial compliance funding as the technology to 2000
1/1 [==============================] - 0s 36ms/step
financial compliance funding as the technology to 2000 on
1/1 [==============================] - 0s 38ms/step
financial compliance funding as the technology to 2000 on 1995
1/1 [==============================] - 0s 36ms/step
financial compliance funding as the technology to 2000 on 1995 he
1/1 [==================

In [ ]:
text = "Computers"
for i in range(15):
  tokenized_text = tokenizer.texts_to_sequences([text])[0]
  padded_token_text = pad_sequences([tokenized_text], maxlen = 114, padding = 'pre')
  max_prob_word_embed = np.argmax(model.predict(padded_token_text))
  for word, index in tokenizer.word_index.items():
      if index == max_prob_word_embed:
         text = text + " " + word
         print(text)

1/1 [==============================] - 0s 52ms/step
Computers with
1/1 [==============================] - 0s 55ms/step
Computers with a
1/1 [==============================] - 0s 56ms/step
Computers with a specific
1/1 [==============================] - 0s 48ms/step
Computers with a specific tape
1/1 [==============================] - 0s 51ms/step
Computers with a specific tape looping
1/1 [==============================] - 0s 51ms/step
Computers with a specific tape looping 5
1/1 [==============================] - 0s 59ms/step
Computers with a specific tape looping 5 and
1/1 [==============================] - 0s 50ms/step
Computers with a specific tape looping 5 and be
1/1 [==============================] - 0s 57ms/step
Computers with a specific tape looping 5 and be used
1/1 [==============================] - 0s 64ms/step
Computers with a specific tape looping 5 and be used in
1/1 [==============================] - 0s 54ms/step
Computers with a specific tape looping 5 and be used in a

In [ ]:
text = "continue"
for i in range(15):
  tokenized_text = tokenizer.texts_to_sequences([text])[0]
  padded_token_text = pad_sequences([tokenized_text], maxlen = 114, padding = 'pre')
  max_prob_word_embed = np.argmax(model.predict(padded_token_text))
  for word, index in tokenizer.word_index.items():
      if index == max_prob_word_embed:
         text = text + " " + word
         print(text)

1/1 [==============================] - 0s 32ms/step
continue to
1/1 [==============================] - 0s 30ms/step
continue to live
1/1 [==============================] - 0s 33ms/step
continue to live underwater
1/1 [==============================] - 0s 34ms/step
continue to live underwater sentient
1/1 [==============================] - 0s 34ms/step
continue to live underwater sentient species
1/1 [==============================] - 0s 36ms/step
continue to live underwater sentient species mysteries
1/1 [==============================] - 0s 34ms/step
continue to live underwater sentient species mysteries every
1/1 [==============================] - 0s 33ms/step
continue to live underwater sentient species mysteries every sentient
1/1 [==============================] - 0s 34ms/step
continue to live underwater sentient species mysteries every sentient species
1/1 [==============================] - 0s 36ms/step
continue to live underwater sentient species mysteries every sentient species

In [ ]:
text = "capable"
for i in range(15):
  tokenized_text = tokenizer.texts_to_sequences([text])[0]
  padded_token_text = pad_sequences([tokenized_text], maxlen = 114, padding = 'pre')
  max_prob_word_embed = np.argmax(model.predict(padded_token_text))
  for word, index in tokenizer.word_index.items():
      if index == max_prob_word_embed:
         text = text + " " + word
         print(text)

1/1 [==============================] - 0s 35ms/step
capable of
1/1 [==============================] - 0s 33ms/step
capable of capodimonte
1/1 [==============================] - 0s 37ms/step
capable of capodimonte naples
1/1 [==============================] - 0s 32ms/step
capable of capodimonte naples that
1/1 [==============================] - 0s 37ms/step
capable of capodimonte naples that same
1/1 [==============================] - 0s 32ms/step
capable of capodimonte naples that same year
1/1 [==============================] - 0s 31ms/step
capable of capodimonte naples that same year he
1/1 [==============================] - 0s 32ms/step
capable of capodimonte naples that same year he coordinates
1/1 [==============================] - 0s 33ms/step
capable of capodimonte naples that same year he coordinates the
1/1 [==============================] - 0s 33ms/step
capable of capodimonte naples that same year he coordinates the work
1/1 [==============================] - 0s 32ms/step
cap

In [ ]:
text = "ideal"
for i in range(15):
  tokenized_text = tokenizer.texts_to_sequences([text])[0]
  padded_token_text = pad_sequences([tokenized_text], maxlen = 114, padding = 'pre')
  max_prob_word_embed = np.argmax(model.predict(padded_token_text))
  for word, index in tokenizer.word_index.items():
      if index == max_prob_word_embed:
         text = text + " " + word
         print(text)

1/1 [==============================] - 0s 34ms/step
ideal for
1/1 [==============================] - 0s 35ms/step
ideal for comparison
1/1 [==============================] - 0s 38ms/step
ideal for comparison shopping
1/1 [==============================] - 0s 33ms/step
ideal for comparison shopping and
1/1 [==============================] - 0s 34ms/step
ideal for comparison shopping and 2d
1/1 [==============================] - 0s 38ms/step
ideal for comparison shopping and 2d barcode
1/1 [==============================] - 0s 37ms/step
ideal for comparison shopping and 2d barcode scanning
1/1 [==============================] - 0s 32ms/step
ideal for comparison shopping and 2d barcode scanning and
1/1 [==============================] - 0s 36ms/step
ideal for comparison shopping and 2d barcode scanning and finding
1/1 [==============================] - 0s 32ms/step
ideal for comparison shopping and 2d barcode scanning and finding information
1/1 [==============================] - 0s 36ms/